In [1]:
#jaccard distance --> bad because ignore ratings
#cosine distance --> treat missing values as negative
#pearson distance/centred cosin distance --> good

In [27]:
from surprise import SVD
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances
from scipy.spatial.distance import correlation

In [28]:
data=pd.read_csv("/home/angps/Documents/DSC3216 (1)/project/the-movies-dataset/ratings_small.csv")
data.head(2)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179


In [29]:
u = data.groupby('userId')['rating'].count()
user = u.sort_values(ascending = False)

m = data.groupby('movieId')['rating'].count()
movie = m.sort_values(ascending = False)

In [30]:
joined = data.join(user, on='userId', how = 'inner', rsuffix='_r')
joined = joined.join(movie, on='movieId', how = 'inner', rsuffix = '_r')

ratings=pd.crosstab(joined.userId, joined.movieId, joined.rating, aggfunc=np.sum)

In [31]:
ratings = ratings.fillna(value=0)
ratings.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
from sklearn import cross_validation as cv
train, test = cv.train_test_split(ratings, test_size=0.20)

In [33]:
test.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
575,3.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
502,4.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
209,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
399,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
train.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
373,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
206,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
126,5.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
171,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
135,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
def distance_train(user1,user2):
        try:
            user1Ratings = train.transpose()[user1]
            user2Ratings = train.transpose()[user2]
            distance = correlation(user1Ratings,user2Ratings)
        except: 
            distance = np.NaN
        return distance

In [36]:
#for test
def distance(testuser,trainuser):
        try:
            user1Ratings = test.transpose()[testuser]
            user2Ratings = train.transpose()[trainuser]
            distance = correlation(user1Ratings,user2Ratings)
        except: 
            distance = np.NaN
        return distance

In [75]:
def get_k_nearest(new_userid,k):
    table = pd.DataFrame(train.index)
    table["dist"] = table["userId"].apply(lambda x: distance(new_userid,x))
    K_nearest_users = table.sort_values(["dist"],ascending=True)[:k]  #get lowest k distance (highest similarity)
    K_nearest_users.set_index("userId",inplace=True)
    return K_nearest_users

def ratings_of_k_nearest(new_userid,k):
    k_nearest = get_k_nearest(new_userid,k)
    rating = train[train.index.isin(k_nearest.index)]
    return rating

def get_ratings(train_user_id,movieid):
    return train.at[train_user_id,movieid]

def pred_ratings(new_userid,k):   #using weighted average
    k_nearest = get_k_nearest(new_userid,k)
    rating = ratings_of_k_nearest(new_userid,k)
    denom = sum(k_nearest["dist"])
    pred = pd.DataFrame(rating.columns)
    pred.set_index("movieId",inplace=True)
    pred["ratings"] = 0
    for movieid in pred.index:
        pred_rating = 0
        for user in k_nearest.index:
            numer = k_nearest.at[user,"dist"]
            weight = numer/denom
            pred_rating += (weight * get_ratings(user,movieid))
        pred.at[movieid,"ratings"] = pred_rating
    pred = pred.sort_values(["ratings"],ascending=False)
    return pred

def recommend_top_n(new_userid,k,top_n):
    if top_n == 0:
        return pred_ratings(new_userid,k).set_index("movieId")
    else:
        return pred_ratings(new_userid,k)[:top_n].set_index("movieId")
            

In [84]:
pred_ratings(575,5)

,ratings
movieId,
954,4
1307,4
260,4
1198,4
1197,4
1097,4
1282,4
953,4
923,4


In [40]:
recommend_top_n(575,5,5)

,ratings
movieId,
290.0,4.797640
595.0,4.607781
1136.0,4.399558
1162.0,4.394667
1484.0,4.310105


In [41]:
def error_rate(testuser,top_k):
    pred = recommend_top_n(testuser,top_k,0).sort_index(ascending=True)
    actual = test[test.index == testuser].T
    return (pred.shape,actual.shape)

In [43]:
error_rate(575,5)

((12490, 1), (9066, 1))

IGNORE BELOW

In [57]:
from sklearn.metrics.pairwise import pairwise_distances
from scipy.spatial.distance import correlation
user_similarity = pairwise_distances(trainmat, metric='cosine')
item_similarity = pairwise_distances(trainmat.T, metric='cosine')

In [58]:
user_similarity

array([[ 0.        ,  0.87726934,  0.95150581, ...,  0.83345575,
         0.94025213,  0.92337011],
       [ 0.87726934,  0.        ,  0.87983023, ...,  0.873132  ,
         0.90852002,  0.86993425],
       [ 0.95150581,  0.87983023,  0.        , ...,  0.98488222,
         0.93976945,  0.9208499 ],
       ..., 
       [ 0.83345575,  0.873132  ,  0.98488222, ...,  0.        ,
         0.83885829,  0.90721596],
       [ 0.94025213,  0.90852002,  0.93976945, ...,  0.83885829,
         0.        ,  0.94809973],
       [ 0.92337011,  0.86993425,  0.9208499 , ...,  0.90721596,
         0.94809973,  0.        ]])

In [39]:
from sklearn.metrics.pairwise import pairwise_distances
from scipy.spatial.distance import correlation
user_similarity = pairwise_distances(trainmat, metric='correlation')
item_similarity = pairwise_distances(trainmat.T, metric='correlation')

In [60]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
sim = 1-spatial.distance.cosine([train.transpose()[98]],[train.transpose()[638]])
sim

0.041535243050478399

In [6]:
sparsity=sum(np.isnan(ratings).sum()/np.prod(ratings.shape))
print ('The sparsity level of data is ' +  str(100-sparsity*100) + '%')

The sparsity level of data is 1.64391416087%


In [24]:
ratings_dict = {'itemID': list(data.movieId),
                'userID': list(data.userId),
                'rating': list(data.rating)}
df = pd.DataFrame(ratings_dict)

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(0.5, 5.0))
# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)

AttributeError: 'DatasetAutoFolds' object has no attribute 'head'

In [26]:
from surprise import Reader, Dataset
from surprise import SVD, evaluate
data.split(n_folds=5)
algo = SVD()
evaluate(algo, data, measures=['RMSE'])

/home/angps/anaconda3/envs/angps/lib/python3.6/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/home/angps/anaconda3/envs/angps/lib/python3.6/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE of algorithm SVD.

------------
Fold 1
RMSE: 0.8913
------------
Fold 2
RMSE: 0.9005
------------
Fold 3
RMSE: 0.8905
------------
Fold 4
RMSE: 0.9045
------------
Fold 5
RMSE: 0.8888
------------
------------
Mean RMSE: 0.8951
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.89128975737373151,
                             0.90052284905442281,
                             0.89048111627707316,
                             0.90449972934953804,
                             0.88884057493554502]})

In [27]:
from surprise import KNNBasic
data.split(n_folds=5)
algo = KNNBasic()
evaluate(algo, data, measures=['RMSE', 'MAE'])

/home/angps/anaconda3/envs/angps/lib/python3.6/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/home/angps/anaconda3/envs/angps/lib/python3.6/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE, MAE of algorithm KNNBasic.

------------
Fold 1
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9686
MAE:  0.7456
------------
Fold 2
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9709
MAE:  0.7458
------------
Fold 3
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9619
MAE:  0.7413
------------
Fold 4
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9634
MAE:  0.7417
------------
Fold 5
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9710
MAE:  0.7464
------------
------------
Mean RMSE: 0.9672
Mean MAE : 0.7441
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'mae': [0.74560126266230309,
                             0.74578044888250627,
                             0.74126837354089803,
                             0.74165303241702973,
                             0.74643505874803273],
                            'rmse': [0.96860283782432355,
                             0.97091929843928637,
                             0.96190090415973473,
                             0.96344740805335816,
                             0.97103438015786414]})